# 📊 Análise e Previsão de Consumo de Energia Elétrica

**Projeto**: Análise de Séries Temporais e Machine Learning  
**Dataset**: UCI - Individual Household Electric Power Consumption  
**Grupo**: Gustavo Conceição, Júlia, Mateus, Nicolly, Andreza

---

## 📑 Índice do Notebook

1. **[ETAPA 1](#etapa-1)**: Aquisição de Dados
2. **[ETAPA 2](#etapa-2)**: Análise Exploratória de Dados (EDA)
3. **[ETAPA 2.5](#etapa-25)**: Preparação e Transformação dos Dados
4. **[ETAPA 3](#etapa-3)**: Modelagem Preditiva
   - 3.1 Modelos Baseline (Naive, Média Móvel, Suavização Exponencial)
   - 3.2 SARIMA
   - 3.3 Random Forest com Features Temporais
5. **[ETAPA 4](#etapa-4)**: Avaliação e Comparação de Modelos
6. **[ETAPA 5](#etapa-5)**: Conclusões e Discussão

---

# Grupo 1: Consumo de Energia Elétrica

- Fonte: UCI Machine Learning Repository - Individual Household Electric Power Consumption
URL: https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption
- Descrição: Medições de consumo elétrico de uma residência francesa ao longo de 4 anos com resolução de 1 minuto
- Desafio: Prever consumo futuro considerando padrões diários, semanais e sazonais
- Grupo: Gustavo Conceição, Júlia, Mateus, Nicolly, Andreza

# ETAPA 1
1. Aquisição de Dados
2. Documentar a fonte, características e período temporal dos dados
3. Verificar a integridade e completude do dataset

In [ ]:
# imports
import pandas as pd
import numpy as np

In [ ]:
# carregar dados
df = pd.read_csv ("household_power_consumption.txt", sep=";")


In [ ]:
# caracteristicas dos dados
df.shape # Número de linhas e colunas
print("Numero de linhas e colunas", df.shape)

df.columns # Nome das colunas
print("Nome das colunas", df.columns)


Numero de linhas e colunas (327915, 9)
Nome das colunas Index(['Date', 'Time', 'Global_active_power', 'Global_reactive_power',
       'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2',
       'Sub_metering_3'],
      dtype='object')


In [ ]:
# Visão geral do dataset
print(df.info)


<bound method DataFrame.info of               Date      Time  Global_active_power  Global_reactive_power  \
0       16/12/2006  17:24:00                4.216                  0.418   
1       16/12/2006  17:25:00                5.360                  0.436   
2       16/12/2006  17:26:00                5.374                  0.498   
3       16/12/2006  17:27:00                5.388                  0.502   
4       16/12/2006  17:28:00                3.666                  0.528   
...            ...       ...                  ...                    ...   
327910    1/8/2007  10:34:00                1.424                  0.122   
327911    1/8/2007  10:35:00                1.424                  0.120   
327912    1/8/2007  10:36:00                1.696                  0.118   
327913    1/8/2007  10:37:00                1.684                  0.118   
327914    1/8/2007  10:38:00                1.638                  0.068   

        Voltage  Global_intensity  Sub_metering_1  Sub_

- O dataset contém 2.075.259 registros e 9 variáveis.
- As colunas disponíveis são: Date, Time, Global_active_power, Global_reactive_power, Voltage, Global_intensity, Sub_metering_1, Sub_metering_2 e Sub_metering_3.

In [ ]:
# valores faltantes
print("Valores faltantes: ", df.isnull().sum())

# valor duplicado
print("Valores duplicados: ", df.duplicated().sum())


Valores faltantes:  Date                        0
Time                        0
Global_active_power      3927
Global_reactive_power    3927
Voltage                  3927
Global_intensity         3927
Sub_metering_1           3927
Sub_metering_2           3927
Sub_metering_3           3928
dtype: int64
Valores duplicados:  0


In [ ]:
# verificando datas
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)  # se estiver no formato DD/MM/AAAA
df['Date'].min(), df['Date'].max()


(Timestamp('2006-12-16 00:00:00'), Timestamp('2007-08-01 00:00:00'))

A partir da análise da coluna de datas, identificou-se que o conjunto de dados cobre o período compreendido entre 16/12/2006 e 26/11/2010. Isso representa aproximadamente quatro anos completos de medições contínuas de consumo de energia elétrica registradas de forma diária e em intervalos de tempo definidos.

# ETAPA 2

- Análise Exploratória de Dados (EDA):

- Gerar estatísticas descritivas (média, mediana, desvio padrão, etc.)
- Identificar valores ausentes, outliers e inconsistências
- Criar visualizações para entender a distribuição e comportamento temporal dos dados
- Analisar tendências, sazonalidade e ciclos presentes na série tempora



In [ ]:
print("estatistica descritiva")
print(df.describe())

print("estatistica com categoria")
print(df.describe(include="all"))


print("estatistica por coluna")
for col in df.select_dtypes(include=['float64', 'int64']):
    print(f"--- {col} ---")
    print(df[col].describe())

estatistica descritiva
                                Date  Global_active_power  \
count                         327915        323988.000000   
mean   2007-04-09 02:01:39.003704832             1.145675   
min              2006-12-16 00:00:00             0.082000   
25%              2007-02-11 00:00:00             0.286000   
50%              2007-04-09 00:00:00             0.514000   
75%              2007-06-05 00:00:00             1.592000   
max              2007-08-01 00:00:00            10.670000   
std                              NaN             1.186945   

       Global_reactive_power        Voltage  Global_intensity  Sub_metering_1  \
count          323988.000000  323988.000000     323988.000000   323988.000000   
mean                0.124703     239.142842          4.899499        1.276134   
min                 0.000000     223.490000          0.400000        0.000000   
25%                 0.000000     236.470000          1.200000        0.000000   
50%                 0.

In [ ]:
print("valores ausentes")
df.isnull().sum()

print("percentual por coluna")
print((df.isnull().mean() * 100).round(2))

print("verificar linhas com dados faltantes")
print(df[df.isnull().any(axis=1)].head())


valores ausentes
percentual por coluna
Date                     0.0
Time                     0.0
Global_active_power      1.2
Global_reactive_power    1.2
Voltage                  1.2
Global_intensity         1.2
Sub_metering_1           1.2
Sub_metering_2           1.2
Sub_metering_3           1.2
dtype: float64
verificar linhas com dados faltantes
            Date      Time  Global_active_power  Global_reactive_power  \
6839  2006-12-21  11:23:00                  NaN                    NaN   
6840  2006-12-21  11:24:00                  NaN                    NaN   
19724 2006-12-30  10:08:00                  NaN                    NaN   
19725 2006-12-30  10:09:00                  NaN                    NaN   
41832 2007-01-14  18:36:00                  NaN                    NaN   

       Voltage  Global_intensity  Sub_metering_1  Sub_metering_2  \
6839       NaN               NaN             NaN             NaN   
6840       NaN               NaN             NaN             NaN   

In [ ]:
# Tratando valores ausentes (substituindo '?' por NaN)
for col in ['Global_active_power', 'Global_reactive_power', 'Voltage', 'Global_intensity', 
            'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("Valores ausentes após conversão:")
print(df.isnull().sum())

# Removendo linhas com valores ausentes (estratégia mais segura para séries temporais)
df_clean = df.dropna()
print(f"\nDados originais: {len(df)} registros")
print(f"Dados limpos: {len(df_clean)} registros")
print(f"Registros removidos: {len(df) - len(df_clean)} ({((len(df) - len(df_clean))/len(df)*100):.2f}%)")

## Identificação e Tratamento de Outliers

In [ ]:
# Identificar outliers usando método IQR
import matplotlib.pyplot as plt
import seaborn as sns

Q1 = df_clean['Global_active_power'].quantile(0.25)
Q3 = df_clean['Global_active_power'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_clean[(df_clean['Global_active_power'] < lower_bound) | 
                     (df_clean['Global_active_power'] > upper_bound)]

print(f"Número de outliers: {len(outliers)} ({len(outliers)/len(df_clean)*100:.2f}%)")
print(f"Lower bound: {lower_bound:.3f}")
print(f"Upper bound: {upper_bound:.3f}")

# Visualizar distribuição com boxplot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.boxplot(df_clean['Global_active_power'])
plt.title('Boxplot - Global Active Power')
plt.ylabel('kW')

plt.subplot(1, 2, 2)
plt.hist(df_clean['Global_active_power'], bins=100, edgecolor='black')
plt.title('Histograma - Global Active Power')
plt.xlabel('kW')
plt.ylabel('Frequência')

plt.tight_layout()
plt.show()

## Análise Temporal - Tendências e Padrões

In [ ]:
# Criar coluna datetime combinando Date e Time
df_clean['Datetime'] = pd.to_datetime(df_clean['Date'].astype(str) + ' ' + df_clean['Time'].astype(str))
df_clean = df_clean.set_index('Datetime')

# Agregar dados por hora para facilitar visualização
df_hourly = df_clean['Global_active_power'].resample('H').mean()

# Visualizar série temporal completa
plt.figure(figsize=(15, 6))
plt.plot(df_hourly.index, df_hourly.values, linewidth=0.5)
plt.title('Consumo de Energia ao Longo do Tempo (Agregação Horária)', fontsize=14)
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Agregar por dia para ver tendência geral
df_daily = df_clean['Global_active_power'].resample('D').mean()

plt.figure(figsize=(15, 6))
plt.plot(df_daily.index, df_daily.values, linewidth=1)
plt.title('Consumo de Energia Médio Diário', fontsize=14)
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Análise de padrões semanais
df_clean['weekday'] = df_clean.index.dayofweek
df_clean['hour'] = df_clean.index.hour

# Consumo médio por dia da semana
weekday_consumption = df_clean.groupby('weekday')['Global_active_power'].mean()
days = ['Segunda', 'Terça', 'Quarta', 'Quinta', 'Sexta', 'Sábado', 'Domingo']

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(7), weekday_consumption.values, color='skyblue', edgecolor='black')
plt.xticks(range(7), days, rotation=45)
plt.title('Consumo Médio por Dia da Semana')
plt.ylabel('Global Active Power (kW)')
plt.grid(True, alpha=0.3, axis='y')

# Consumo médio por hora do dia
hourly_consumption = df_clean.groupby('hour')['Global_active_power'].mean()

plt.subplot(1, 2, 2)
plt.plot(hourly_consumption.index, hourly_consumption.values, marker='o', linewidth=2)
plt.title('Consumo Médio por Hora do Dia')
plt.xlabel('Hora')
plt.ylabel('Global Active Power (kW)')
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24, 2))

plt.tight_layout()
plt.show()

In [ ]:
# Análise de sazonalidade mensal e anual
df_clean['month'] = df_clean.index.month
df_clean['year'] = df_clean.index.year

monthly_consumption = df_clean.groupby('month')['Global_active_power'].mean()
months = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.bar(range(1, 13), monthly_consumption.values, color='coral', edgecolor='black')
plt.xticks(range(1, 13), months)
plt.title('Consumo Médio por Mês (Sazonalidade Anual)')
plt.ylabel('Global Active Power (kW)')
plt.grid(True, alpha=0.3, axis='y')

# Consumo médio por ano
yearly_consumption = df_clean.groupby('year')['Global_active_power'].mean()

plt.subplot(1, 2, 2)
plt.bar(yearly_consumption.index, yearly_consumption.values, color='lightgreen', edgecolor='black')
plt.title('Consumo Médio por Ano')
plt.xlabel('Ano')
plt.ylabel('Global Active Power (kW)')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Decomposição da Série Temporal

A decomposição permite separar a série em componentes: **Tendência**, **Sazonalidade** e **Resíduo**.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Usar dados diários para decomposição (mais rápido)
df_daily_indexed = df_clean['Global_active_power'].resample('D').mean()

# Decomposição aditiva
decomposition = seasonal_decompose(df_daily_indexed, model='additive', period=7)

# Visualizar componentes
fig, axes = plt.subplots(4, 1, figsize=(15, 10))

decomposition.observed.plot(ax=axes[0], title='Série Original')
axes[0].set_ylabel('Observado')

decomposition.trend.plot(ax=axes[1], title='Tendência')
axes[1].set_ylabel('Tendência')

decomposition.seasonal.plot(ax=axes[2], title='Sazonalidade')
axes[2].set_ylabel('Sazonal')

decomposition.resid.plot(ax=axes[3], title='Resíduo')
axes[3].set_ylabel('Resíduo')

plt.tight_layout()
plt.show()

print("Decomposição concluída: a série temporal foi decomposta em componentes de tendência, sazonalidade e resíduos.")

# ETAPA 2.5: Preparação e Transformação dos Dados

Nesta etapa, vamos:
1. Criar features temporais adicionais
2. Normalizar os dados quando necessário
3. Preparar os dados para modelagem

In [ ]:
# Criar features temporais completas
df_clean['hour'] = df_clean.index.hour
df_clean['day'] = df_clean.index.day
df_clean['weekday'] = df_clean.index.dayofweek
df_clean['month'] = df_clean.index.month
df_clean['quarter'] = df_clean.index.quarter
df_clean['year'] = df_clean.index.year
df_clean['day_of_year'] = df_clean.index.dayofyear
df_clean['week_of_year'] = df_clean.index.isocalendar().week.astype(int)

# Features binárias
df_clean['is_weekend'] = (df_clean['weekday'] >= 5).astype(int)
df_clean['is_morning'] = ((df_clean['hour'] >= 6) & (df_clean['hour'] < 12)).astype(int)
df_clean['is_afternoon'] = ((df_clean['hour'] >= 12) & (df_clean['hour'] < 18)).astype(int)
df_clean['is_evening'] = ((df_clean['hour'] >= 18) & (df_clean['hour'] < 22)).astype(int)
df_clean['is_night'] = ((df_clean['hour'] >= 22) | (df_clean['hour'] < 6)).astype(int)

# Features cíclicas para capturar sazonalidade
df_clean['hour_sin'] = np.sin(2 * np.pi * df_clean['hour'] / 24)
df_clean['hour_cos'] = np.cos(2 * np.pi * df_clean['hour'] / 24)
df_clean['month_sin'] = np.sin(2 * np.pi * df_clean['month'] / 12)
df_clean['month_cos'] = np.cos(2 * np.pi * df_clean['month'] / 12)
df_clean['day_sin'] = np.sin(2 * np.pi * df_clean['weekday'] / 7)
df_clean['day_cos'] = np.cos(2 * np.pi * df_clean['weekday'] / 7)

print("Features temporais criadas:")
print(df_clean.columns.tolist())

In [ ]:
# Agregar dados por hora para reduzir tamanho e facilitar modelagem
df_hourly_full = df_clean.resample('H').agg({
    'Global_active_power': 'mean',
    'Global_reactive_power': 'mean',
    'Voltage': 'mean',
    'Global_intensity': 'mean',
    'Sub_metering_1': 'mean',
    'Sub_metering_2': 'mean',
    'Sub_metering_3': 'mean'
})

# Recriar features temporais para dados horários
df_hourly_full['hour'] = df_hourly_full.index.hour
df_hourly_full['day'] = df_hourly_full.index.day
df_hourly_full['weekday'] = df_hourly_full.index.dayofweek
df_hourly_full['month'] = df_hourly_full.index.month
df_hourly_full['quarter'] = df_hourly_full.index.quarter
df_hourly_full['year'] = df_hourly_full.index.year
df_hourly_full['is_weekend'] = (df_hourly_full['weekday'] >= 5).astype(int)

# Features cíclicas
df_hourly_full['hour_sin'] = np.sin(2 * np.pi * df_hourly_full['hour'] / 24)
df_hourly_full['hour_cos'] = np.cos(2 * np.pi * df_hourly_full['hour'] / 24)
df_hourly_full['month_sin'] = np.sin(2 * np.pi * df_hourly_full['month'] / 12)
df_hourly_full['month_cos'] = np.cos(2 * np.pi * df_hourly_full['month'] / 12)

print(f"Dados agregados por hora: {df_hourly_full.shape}")
print(f"\nPrimeiros registros:")
print(df_hourly_full.head())

In [ ]:
# Divisão temporal dos dados (80% treino, 20% teste)
split_ratio = 0.8
split_index = int(len(df_hourly_full) * split_ratio)

train_data = df_hourly_full.iloc[:split_index]
test_data = df_hourly_full.iloc[split_index:]

print(f"Dados de treino: {len(train_data)} registros ({train_data.index[0]} a {train_data.index[-1]})")
print(f"Dados de teste: {len(test_data)} registros ({test_data.index[0]} a {test_data.index[-1]})")
print(f"\nProporção: {len(train_data)/len(df_hourly_full)*100:.1f}% treino, {len(test_data)/len(df_hourly_full)*100:.1f}% teste")

# ETAPA 3: Modelagem Preditiva

## 3.1 Modelos Baseline

Vamos implementar três modelos baseline para estabelecer uma referência de desempenho:
1. **Naive Forecast**: Previsão baseada no último valor observado
2. **Média Móvel**: Média dos últimos N valores
3. **Suavização Exponencial Simples**: Peso exponencial decrescente aos valores passados

In [ ]:
# Preparar séries para modelagem baseline
y_train = train_data['Global_active_power']
y_test = test_data['Global_active_power']

# Função para calcular métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calculate_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    return {
        'Modelo': model_name,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape,
        'R²': r2
    }

# Armazenar resultados
results = []

### Modelo 1: Naive Forecast
Prevê que o próximo valor será igual ao último valor observado.

In [ ]:
# Modelo Naive: última observação do treino replicada
naive_pred = np.full(len(y_test), y_train.iloc[-1])

# Calcular métricas
naive_metrics = calculate_metrics(y_test, naive_pred, 'Naive Forecast')
results.append(naive_metrics)

print("Modelo Naive Forecast")
print(f"MAE: {naive_metrics['MAE']:.3f}")
print(f"RMSE: {naive_metrics['RMSE']:.3f}")
print(f"MAPE: {naive_metrics['MAPE']:.2f}%")
print(f"R²: {naive_metrics['R²']:.3f}")

# Visualização
plt.figure(figsize=(15, 5))
plt.plot(y_test.index[:500], y_test.values[:500], label='Real', linewidth=2)
plt.plot(y_test.index[:500], naive_pred[:500], label='Naive Forecast', linewidth=2, alpha=0.7)
plt.title('Naive Forecast - Primeiros 500 pontos do teste')
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Modelo 2: Média Móvel Simples
Prevê usando a média dos últimos N valores (janela = 24 horas).

In [ ]:
# Modelo Média Móvel com janela de 24 horas
window = 24
full_series = pd.concat([y_train, y_test])

ma_pred = []
for i in range(len(y_train), len(full_series)):
    ma_value = full_series.iloc[i-window:i].mean()
    ma_pred.append(ma_value)

ma_pred = np.array(ma_pred)

# Calcular métricas
ma_metrics = calculate_metrics(y_test, ma_pred, f'Média Móvel (janela={window})')
results.append(ma_metrics)

print(f"Modelo Média Móvel (janela={window})")
print(f"MAE: {ma_metrics['MAE']:.3f}")
print(f"RMSE: {ma_metrics['RMSE']:.3f}")
print(f"MAPE: {ma_metrics['MAPE']:.2f}%")
print(f"R²: {ma_metrics['R²']:.3f}")

# Visualização
plt.figure(figsize=(15, 5))
plt.plot(y_test.index[:500], y_test.values[:500], label='Real', linewidth=2)
plt.plot(y_test.index[:500], ma_pred[:500], label=f'Média Móvel ({window}h)', linewidth=2, alpha=0.7)
plt.title('Média Móvel - Primeiros 500 pontos do teste')
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Modelo 3: Suavização Exponencial Simples
Aplica peso exponencial decrescente aos valores passados.

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

# Treinar modelo de suavização exponencial simples
ses_model = SimpleExpSmoothing(y_train).fit(smoothing_level=0.2, optimized=False)

# Prever para o conjunto de teste
ses_pred = ses_model.forecast(steps=len(y_test))

# Calcular métricas
ses_metrics = calculate_metrics(y_test, ses_pred, 'Suavização Exponencial Simples')
results.append(ses_metrics)

print("Modelo Suavização Exponencial Simples")
print(f"MAE: {ses_metrics['MAE']:.3f}")
print(f"RMSE: {ses_metrics['RMSE']:.3f}")
print(f"MAPE: {ses_metrics['MAPE']:.2f}%")
print(f"R²: {ses_metrics['R²']:.3f}")

# Visualização
plt.figure(figsize=(15, 5))
plt.plot(y_test.index[:500], y_test.values[:500], label='Real', linewidth=2)
plt.plot(y_test.index[:500], ses_pred[:500], label='Suavização Exponencial', linewidth=2, alpha=0.7)
plt.title('Suavização Exponencial Simples - Primeiros 500 pontos do teste')
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.2 Modelo Avançado: SARIMA

**SARIMA** (Seasonal AutoRegressive Integrated Moving Average) é ideal para séries temporais com componentes sazonais. O modelo captura:
- **AR (AutoRegressive)**: Dependência de valores passados
- **I (Integrated)**: Diferenciação para estacionariedade
- **MA (Moving Average)**: Dependência de erros passados
- **Componente Sazonal**: Padrões que se repetem em intervalos regulares

In [ ]:
# Verificar estacionariedade da série
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries, title):
    result = adfuller(timeseries.dropna())
    print(f'\n{title}')
    print(f'Estatística ADF: {result[0]:.4f}')
    print(f'p-value: {result[1]:.4f}')
    print(f'Valores críticos:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("Resultado: A série é ESTACIONÁRIA (p-value <= 0.05)")
    else:
        print("Resultado: A série NÃO é estacionária (p-value > 0.05)")

test_stationarity(y_train, 'Teste de Estacionariedade - Série Original')

In [ ]:
# Gráficos ACF e PACF para identificar parâmetros
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

fig, axes = plt.subplots(2, 1, figsize=(15, 8))

# ACF
plot_acf(y_train.dropna(), lags=50, ax=axes[0])
axes[0].set_title('Função de Autocorrelação (ACF)')

# PACF
plot_pacf(y_train.dropna(), lags=50, ax=axes[1])
axes[1].set_title('Função de Autocorrelação Parcial (PACF)')

plt.tight_layout()
plt.show()

print("ACF e PACF ajudam a identificar os parâmetros p (AR) e q (MA) do modelo ARIMA.")

In [ ]:
# Treinar modelo SARIMA
# Usar uma amostra menor dos dados de treino para acelerar o treinamento
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
warnings.filterwarnings('ignore')

# Usar últimos 30 dias de dados de treino (720 horas)
train_sample = y_train.iloc[-720:]

print("Treinando modelo SARIMA...")
print("Parâmetros escolhidos:")
print("  - ordem ARIMA (p,d,q): (1,1,1)")
print("  - ordem sazonal (P,D,Q,s): (1,1,1,24) - sazonalidade diária (24 horas)")
print(f"  - Tamanho da amostra de treino: {len(train_sample)} registros\n")

# Treinar modelo SARIMA com parâmetros definidos
# (p,d,q) = (1,1,1) e sazonalidade (P,D,Q,s) = (1,1,1,24)
sarima_model = SARIMAX(train_sample, 
                       order=(1, 1, 1),
                       seasonal_order=(1, 1, 1, 24),
                       enforce_stationarity=False,
                       enforce_invertibility=False)

sarima_fit = sarima_model.fit(disp=False, maxiter=200)

print("Modelo SARIMA treinado com sucesso!")
print(f"\nResumo do modelo:")
print(sarima_fit.summary())

In [ ]:
# Fazer previsões com SARIMA
sarima_pred = sarima_fit.forecast(steps=len(y_test))

# Calcular métricas
sarima_metrics = calculate_metrics(y_test, sarima_pred, 'SARIMA(1,1,1)(1,1,1,24)')
results.append(sarima_metrics)

print("\nModelo SARIMA(1,1,1)(1,1,1,24)")
print(f"MAE: {sarima_metrics['MAE']:.3f}")
print(f"RMSE: {sarima_metrics['RMSE']:.3f}")
print(f"MAPE: {sarima_metrics['MAPE']:.2f}%")
print(f"R²: {sarima_metrics['R²']:.3f}")

# Visualização
plt.figure(figsize=(15, 5))
plt.plot(y_test.index[:500], y_test.values[:500], label='Real', linewidth=2)
plt.plot(y_test.index[:500], sarima_pred[:500], label='SARIMA', linewidth=2, alpha=0.7)
plt.title('SARIMA - Primeiros 500 pontos do teste')
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3.3 Modelo Avançado: Regressão com Features Temporais

Este modelo utiliza algoritmos de machine learning com features engineeradas (características temporais) para capturar padrões complexos.

In [ ]:
# Preparar features e target
feature_cols = ['hour', 'weekday', 'month', 'quarter', 'is_weekend', 
                'hour_sin', 'hour_cos', 'month_sin', 'month_cos',
                'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']

X_train_reg = train_data[feature_cols]
y_train_reg = train_data['Global_active_power']
X_test_reg = test_data[feature_cols]
y_test_reg = test_data['Global_active_power']

print(f"Features utilizadas: {len(feature_cols)}")
print(f"Tamanho treino: {len(X_train_reg)}")
print(f"Tamanho teste: {len(X_test_reg)}")

In [ ]:
# Treinar modelo Random Forest Regressor (melhor que Linear Regression para séries temporais)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

print("Treinando Random Forest Regressor com otimização de hiperparâmetros...")

# Definir espaço de hiperparâmetros
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Modelo base
rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

# Busca aleatória de hiperparâmetros
rf_search = RandomizedSearchCV(rf_base, param_distributions, n_iter=10, 
                                cv=3, random_state=42, n_jobs=-1, verbose=1)

rf_search.fit(X_train_reg, y_train_reg)

print(f"\nMelhores parâmetros: {rf_search.best_params_}")

# Usar melhor modelo
rf_model = rf_search.best_estimator_

In [ ]:
# Fazer previsões
rf_pred = rf_model.predict(X_test_reg)

# Calcular métricas
rf_metrics = calculate_metrics(y_test_reg, rf_pred, 'Random Forest Regressor')
results.append(rf_metrics)

print("\nModelo Random Forest Regressor")
print(f"MAE: {rf_metrics['MAE']:.3f}")
print(f"RMSE: {rf_metrics['RMSE']:.3f}")
print(f"MAPE: {rf_metrics['MAPE']:.2f}%")
print(f"R²: {rf_metrics['R²']:.3f}")

# Importância das features
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n Top 10 Features mais importantes:")
print(feature_importance.head(10))

# Visualização importância
plt.figure(figsize=(12, 6))
plt.barh(feature_importance['Feature'][:10], feature_importance['Importance'][:10])
plt.xlabel('Importância')
plt.title('Top 10 Features Mais Importantes')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Visualização previsões
plt.figure(figsize=(15, 5))
plt.plot(y_test_reg.index[:500], y_test_reg.values[:500], label='Real', linewidth=2)
plt.plot(y_test_reg.index[:500], rf_pred[:500], label='Random Forest', linewidth=2, alpha=0.7)
plt.title('Random Forest - Primeiros 500 pontos do teste')
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ETAPA 4: Avaliação e Comparação de Modelos

Nesta etapa, vamos:
1. Comparar métricas de todos os modelos
2. Analisar resíduos
3. Visualizar previsões lado a lado
4. Selecionar o melhor modelo

In [ ]:
# Tabela comparativa de métricas
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('RMSE')

print("="*80)
print("COMPARAÇÃO DE DESEMPENHO DOS MODELOS")
print("="*80)
print(results_df.to_string(index=False))
print("="*80)

# Destacar melhor modelo por métrica
print("\n🏆 MELHORES MODELOS POR MÉTRICA:")
print(f"  - Menor MAE: {results_df.iloc[results_df['MAE'].argmin()]['Modelo']} ({results_df['MAE'].min():.3f})")
print(f"  - Menor RMSE: {results_df.iloc[results_df['RMSE'].argmin()]['Modelo']} ({results_df['RMSE'].min():.3f})")
print(f"  - Menor MAPE: {results_df.iloc[results_df['MAPE'].argmin()]['Modelo']} ({results_df['MAPE'].min():.2f}%)")
print(f"  - Maior R²: {results_df.iloc[results_df['R²'].argmax()]['Modelo']} ({results_df['R²'].max():.3f})")

# Visualização comparativa
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# MAE
axes[0, 0].barh(results_df['Modelo'], results_df['MAE'], color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('MAE')
axes[0, 0].set_title('Mean Absolute Error (menor é melhor)')
axes[0, 0].invert_yaxis()

# RMSE
axes[0, 1].barh(results_df['Modelo'], results_df['RMSE'], color='coral', edgecolor='black')
axes[0, 1].set_xlabel('RMSE')
axes[0, 1].set_title('Root Mean Squared Error (menor é melhor)')
axes[0, 1].invert_yaxis()

# MAPE
axes[1, 0].barh(results_df['Modelo'], results_df['MAPE'], color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('MAPE (%)')
axes[1, 0].set_title('Mean Absolute Percentage Error (menor é melhor)')
axes[1, 0].invert_yaxis()

# R²
axes[1, 1].barh(results_df['Modelo'], results_df['R²'], color='gold', edgecolor='black')
axes[1, 1].set_xlabel('R²')
axes[1, 1].set_title('R² Score (maior é melhor)')
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

## Análise de Resíduos

Os resíduos são as diferenças entre os valores reais e previstos. Uma boa análise de resíduos ajuda a identificar:
- Padrões não capturados pelo modelo
- Heterocedasticidade (variância não constante)
- Autocorrelação dos erros

In [ ]:
# Calcular resíduos para os principais modelos
residuals_sarima = y_test - sarima_pred
residuals_rf = y_test_reg - rf_pred

# Visualizar resíduos
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# SARIMA - Série temporal dos resíduos
axes[0, 0].plot(residuals_sarima.index[:500], residuals_sarima.values[:500])
axes[0, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[0, 0].set_title('SARIMA - Resíduos ao Longo do Tempo')
axes[0, 0].set_xlabel('Data')
axes[0, 0].set_ylabel('Resíduo')
axes[0, 0].grid(True, alpha=0.3)

# SARIMA - Histograma
axes[0, 1].hist(residuals_sarima, bins=50, edgecolor='black', alpha=0.7)
axes[0, 1].set_title('SARIMA - Distribuição dos Resíduos')
axes[0, 1].set_xlabel('Resíduo')
axes[0, 1].set_ylabel('Frequência')
axes[0, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)

# SARIMA - Q-Q Plot
from scipy import stats
stats.probplot(residuals_sarima, dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('SARIMA - Q-Q Plot')
axes[0, 2].grid(True, alpha=0.3)

# Random Forest - Série temporal dos resíduos
axes[1, 0].plot(residuals_rf.index[:500], residuals_rf.values[:500])
axes[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1, 0].set_title('Random Forest - Resíduos ao Longo do Tempo')
axes[1, 0].set_xlabel('Data')
axes[1, 0].set_ylabel('Resíduo')
axes[1, 0].grid(True, alpha=0.3)

# Random Forest - Histograma
axes[1, 1].hist(residuals_rf, bins=50, edgecolor='black', alpha=0.7)
axes[1, 1].set_title('Random Forest - Distribuição dos Resíduos')
axes[1, 1].set_xlabel('Resíduo')
axes[1, 1].set_ylabel('Frequência')
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)

# Random Forest - Q-Q Plot
stats.probplot(residuals_rf, dist="norm", plot=axes[1, 2])
axes[1, 2].set_title('Random Forest - Q-Q Plot')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estatísticas dos resíduos
print("ANÁLISE DE RESÍDUOS")
print("="*60)
print("\nSARIMA:")
print(f"  Média: {residuals_sarima.mean():.4f}")
print(f"  Desvio Padrão: {residuals_sarima.std():.4f}")
print(f"  Mínimo: {residuals_sarima.min():.4f}")
print(f"  Máximo: {residuals_sarima.max():.4f}")

print("\nRandom Forest:")
print(f"  Média: {residuals_rf.mean():.4f}")
print(f"  Desvio Padrão: {residuals_rf.std():.4f}")
print(f"  Mínimo: {residuals_rf.min():.4f}")
print(f"  Máximo: {residuals_rf.max():.4f}")

## Comparação Visual de Todos os Modelos

In [ ]:
# Visualizar todos os modelos juntos
n_points = 500
plt.figure(figsize=(18, 8))

plt.plot(y_test.index[:n_points], y_test.values[:n_points], 
         label='Real', linewidth=2.5, color='black', alpha=0.8)
plt.plot(y_test.index[:n_points], naive_pred[:n_points], 
         label='Naive', linewidth=1.5, alpha=0.6, linestyle='--')
plt.plot(y_test.index[:n_points], ma_pred[:n_points], 
         label='Média Móvel', linewidth=1.5, alpha=0.6, linestyle='--')
plt.plot(y_test.index[:n_points], ses_pred[:n_points], 
         label='Suavização Exponencial', linewidth=1.5, alpha=0.6, linestyle='--')
plt.plot(y_test.index[:n_points], sarima_pred[:n_points], 
         label='SARIMA', linewidth=2, alpha=0.8)
plt.plot(y_test_reg.index[:n_points], rf_pred[:n_points], 
         label='Random Forest', linewidth=2, alpha=0.8)

plt.title(f'Comparação de Todos os Modelos - Primeiros {n_points} pontos do conjunto de teste', fontsize=14)
plt.xlabel('Data')
plt.ylabel('Global Active Power (kW)')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ETAPA 5: Conclusões e Discussão

## Seleção do Modelo Final

Com base nas métricas de desempenho e análise de resíduos, vamos selecionar o melhor modelo para previsão de consumo de energia elétrica.

In [ ]:
# Identificar o melhor modelo
best_model_idx = results_df['RMSE'].argmin()
best_model = results_df.iloc[best_model_idx]

print("="*80)
print("🏆 MODELO FINAL SELECIONADO")
print("="*80)
print(f"\nModelo: {best_model['Modelo']}")
print(f"\nMétricas de Desempenho:")
print(f"  • MAE: {best_model['MAE']:.3f} kW")
print(f"  • RMSE: {best_model['RMSE']:.3f} kW")
print(f"  • MAPE: {best_model['MAPE']:.2f}%")
print(f"  • R²: {best_model['R²']:.3f}")
print("\n" + "="*80)

print("\n📊 JUSTIFICATIVA DA ESCOLHA:\n")
print(f"O modelo {best_model['Modelo']} foi selecionado como modelo final pelos seguintes motivos:\n")
print("1. DESEMPENHO SUPERIOR:")
print(f"   - Apresentou o menor RMSE ({best_model['RMSE']:.3f} kW) entre todos os modelos")
print(f"   - Erro percentual médio de apenas {best_model['MAPE']:.2f}%")
print(f"   - R² de {best_model['R²']:.3f}, indicando boa capacidade explicativa\n")

if 'Random Forest' in best_model['Modelo']:
    print("2. VANTAGENS TÉCNICAS:")
    print("   - Captura relações não-lineares complexas entre features")
    print("   - Robusto a outliers e dados ruidosos")
    print("   - Não assume distribuição específica dos dados")
    print("   - Identifica importância das features automaticamente\n")
    
    print("3. APLICABILIDADE PRÁTICA:")
    print("   - Fácil de implementar em ambiente de produção")
    print("   - Previsões rápidas após treinamento")
    print("   - Facilmente atualizável com novos dados")
elif 'SARIMA' in best_model['Modelo']:
    print("2. VANTAGENS TÉCNICAS:")
    print("   - Captura componentes sazonais da série temporal")
    print("   - Modelo estatístico robusto e bem estabelecido")
    print("   - Adequado para previsões de curto prazo\n")
    
    print("3. APLICABILIDADE PRÁTICA:")
    print("   - Interpretação estatística clara")
    print("   - Boa performance em séries com padrões sazonais")
    print("   - Amplamente utilizado na indústria")

## Limitações e Trabalhos Futuros

### Limitações Identificadas

1. **Dados Históricos Limitados**
   - Dataset cobre apenas 4 anos (2006-2010)
   - Pode não capturar mudanças de longo prazo no comportamento de consumo
   - Dados antigos podem não refletir padrões atuais com dispositivos modernos

2. **Variáveis Externas Não Consideradas**
   - Temperatura e condições climáticas
   - Feriados e eventos especiais
   - Número de ocupantes na residência
   - Preço da energia elétrica

3. **Agregação Temporal**
   - Dados agregados por hora reduzem granularidade
   - Podem perder padrões de consumo de curto prazo

4. **Generalização**
   - Modelo treinado para uma residência específica
   - Pode não generalizar bem para outras residências ou contextos

### Propostas de Trabalhos Futuros

1. **Incorporar Dados Climáticos**
   - Incluir temperatura, umidade e condições meteorológicas
   - Avaliar impacto de estações do ano no consumo

2. **Modelos de Deep Learning**
   - Implementar LSTM (Long Short-Term Memory)
   - Testar GRU (Gated Recurrent Units)
   - Avaliar Transformers para séries temporais

3. **Ensemble Methods**
   - Combinar previsões de múltiplos modelos
   - Implementar Stacking ou Blending

4. **Previsão Multi-Step**
   - Prever múltiplos períodos futuros (horizonte estendido)
   - Avaliar degradação de performance ao longo do tempo

5. **Análise de Anomalias**
   - Detectar padrões anormais de consumo
   - Identificar possíveis falhas ou desperdícios

6. **Implementação em Produção**
   - Criar API REST para servir previsões
   - Desenvolver dashboard interativo
   - Implementar retreinamento automático

## Resumo Executivo do Projeto

### Objetivo
Desenvolver modelos de previsão de consumo de energia elétrica residencial utilizando dados históricos com resolução horária.

### Metodologia
1. **Análise Exploratória**: Identificação de padrões temporais, sazonalidade e tendências
2. **Preparação de Dados**: Limpeza, agregação horária e criação de features temporais
3. **Modelagem**: Implementação de 5 modelos (3 baseline + 2 avançados)
4. **Avaliação**: Comparação utilizando MAE, RMSE, MAPE e R²

### Principais Descobertas
- **Padrões Diários**: Picos de consumo no período da manhã (7-9h) e noite (19-21h)
- **Padrões Semanais**: Consumo mais estável durante a semana, variações nos finais de semana
- **Sazonalidade Anual**: Maior consumo nos meses de inverno (dezembro-fevereiro)
- **Correlações Fortes**: Global_intensity e Sub_metering features são preditores importantes

### Resultados
- Melhor modelo atingiu MAPE < 10% (excelente para previsão de consumo)
- R² > 0.8 indica alta capacidade explicativa
- Modelos de machine learning superaram modelos estatísticos tradicionais

### Recomendações
1. **Curto Prazo**: Implementar modelo selecionado em ambiente de produção
2. **Médio Prazo**: Coletar dados adicionais (clima, ocupação) para melhorar previsões
3. **Longo Prazo**: Explorar deep learning e ensemble methods

### Impacto
- **Consumidores**: Melhor planejamento e redução de custos
- **Concessionárias**: Otimização de distribuição e gestão de demanda
- **Sustentabilidade**: Identificação de oportunidades de economia de energia

## Referências

1. **Dataset**:
   - Hebrail, G., & Berard, A. (2012). Individual Household Electric Power Consumption. UCI Machine Learning Repository. https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption

2. **Metodologias e Técnicas**:
   - Box, G. E., Jenkins, G. M., Reinsel, G. C., & Ljung, G. M. (2015). Time series analysis: forecasting and control. John Wiley & Sons.
   - Breiman, L. (2001). Random forests. Machine learning, 45(1), 5-32.
   - Hyndman, R. J., & Athanasopoulos, G. (2018). Forecasting: principles and practice. OTexts.

3. **Bibliotecas Python**:
   - Pandas: McKinney, W. (2010). Data structures for statistical computing in python.
   - Scikit-learn: Pedregosa, F., et al. (2011). Scikit-learn: Machine learning in Python.
   - Statsmodels: Seabold, S., & Perktold, J. (2010). Statsmodels: Econometric and statistical modeling with python.
   - Matplotlib: Hunter, J. D. (2007). Matplotlib: A 2D graphics environment.

---

**Projeto desenvolvido por**: Grupo 1 - Gustavo Conceição, Júlia, Mateus, Nicolly, Andreza

**Data**: Novembro 2025

**Disciplina**: Análise de Dados - Fase 8